<a href="https://colab.research.google.com/github/danebencedavid/NLP-A-Agent/blob/master/agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import numpy as np
import faiss
import os
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import gradio as gr

In [27]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
model = AutoModel.from_pretrained("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [6]:
embeddings_folder = "/content/drive/MyDrive/full_abs_embeddings"

In [7]:
full_df = pd.read_csv('/content/drive/MyDrive/fulldf_papers.csv')

In [12]:
full_df['abstract'] = full_df['abstract'].fillna('')
non_empty_df = full_df[full_df['abstract'].str.strip() != ''].copy()
f_abstracts = non_empty_df['abstract'].tolist()
print(f"Number of non-empty abstracts: {len(f_abstracts)}")

Number of non-empty abstracts: 35


In [13]:
full_df = non_empty_df

In [16]:
print(len(full_df))

35


In [17]:
embedding_files = sorted([f for f in os.listdir(embeddings_folder) if f.endswith(".npy")])
all_embeddings = [np.load(os.path.join(embeddings_folder, f)) for f in embedding_files]
embeddings = np.concatenate(all_embeddings, axis=0)
print(f"Loaded {embeddings.shape[0]} embeddings.")

Loaded 35 embeddings.


In [18]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(embeddings.astype(np.float32))

In [19]:
faiss_index_path = os.path.join(embeddings_folder, "faiss_index.index")
faiss.write_index(index, faiss_index_path)

print(f"Faiss index created and saved at {faiss_index_path}")

Faiss index created and saved at /content/drive/MyDrive/full_abs_embeddings/faiss_index.index


In [20]:
faiss_index_path = os.path.join(embeddings_folder, "faiss_index.index")
index = faiss.read_index(faiss_index_path)

In [25]:
def get_relevant_papers(query, index, embeddings, k=5):
    query_embedding = model.encode([query])

    distances, indices = index.search(query_embedding, k)

    relevant_papers = []
    for i in indices[0]:
        relevant_papers.append(full_df.iloc[i]['abstract'])

    return relevant_papers, distances[0]

In [26]:
def answer_query(query):
    relevant_papers, distances = get_relevant_papers(query, index, embeddings, k=3)
    return "\n\n".join([f"Paper {i+1}: {relevant_papers[i]} (Distance: {distances[i]:.3f})" for i in range(len(relevant_papers))])